In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

In [ ]:
receiver_vector = fj.get('vector', name='receiver1')

In [ ]:
study = fj.create('study', name='Loica testing', description='Test study for demonstrating Loica')

In [ ]:
dna = fj.create('dna', name='not')
vector = fj.create('vector', name='not', dnas=dna.id)

In [ ]:
network = GeneticNetwork(vector=vector.id[0])

In [ ]:
sfp = fj.create('signal', name='SFP', color='green', description='Simulated fluorescent protein')
reporter = Reporter(name='SFP', color='green', degradation_rate=0, init_concentration=0, signal_id=sfp.id[0])

In [ ]:
network.add_reporter(reporter)

The Not operator is a device which is repressed by a single repressor $r$, and produces an output expression rate $\phi(r)$ modeled as follows:

\begin{equation}
    \phi(s)
    =
    \frac
    {
        a + b (\frac{r}{K})^n
    }
    {
        1 + (\frac{r}{K})^n
    }
\end{equation}

In [ ]:
ahl = Supplement(name='AHL')
repressor = Regulator('LacI')
rec = Receiver(input=ahl, output=repressor, a=0, b=1, K=1, n=2)
not_ = Not(input=repressor, output=reporter, a=1, b=0, K=1, n=2)
network.add_operators([rec,not_])
network.add_regulator(repressor)

In [ ]:
network.operators

In [ ]:
plt.figure(figsize=(3,3), dpi=150)
network.draw()

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.05, 1, 1, 1)

def biomass(t):
    return gompertz(t, 0.05, 1, 1, 1)
    
metab = SimulatedMetabolism(biomass, growth_rate)

media = fj.create('media', name='loica', description='Simulated loica media')
strain = fj.create('strain', name='loica', description='Loica test strain')

In [ ]:
# Create list of samples    
samples = []
concs = np.logspace(-6, 6, 24)
for conc in concs:
    sample = Sample(geneetic_network=network, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
    # Add AHL to samples at given concentration
    sample.add_supplement(ahl, conc)
    samples.append(sample)
    
# Add sample with no inducer
sample = Sample(circuit=network, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
# Add AHL to samples at given concentration
sample.add_supplement(ahl, 0)
samples.append(sample)

In [ ]:
biomass_signal = fj.create('signal', name='SOD', description='Simulated OD', color='black')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica inverter',
              description='Simulated inverter generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run(nsr=0)

In [ ]:
assay.upload(fj, study.id[0])

In [ ]:
not_.characterize(
    fj,
    receiver=receiver_vector.id,
    inverter=vector.id,
    media=media.id,
    strain=strain.id,
    signal=sfp.id,
    biomass_signal=biomass_signal.id,
    gamma=0
)

In [ ]:
not_.a, not_.b, not_.K, not_.n

In [ ]:
not_.a_A, not_.b_A, not_.K_A, not_.n_A